In [6]:
using JuMP
using HiGHS
using Ipopt
using Optimization

In [ ]:
function leer_archivo_tsp(archivo)
    coordenadas = []
    open(archivo, "r") do f
        en_seccion = false
        for linea in eachline(f)
            if occursin("NODE_COORD_SECTION", linea)
                en_seccion = true
                continue
            elseif occursin("EOF", linea)
                break
            end
            
            if en_seccion
                partes = split(linea)
                if length(partes) >= 3
                    id = parse(Int, partes[1])
                    lat = parse(Float64, partes[2])
                    lon = parse(Float64, partes[3])
                    push!(coordenadas, (id, lat, lon))
                end
            end
        end
    end
    return coordenadas
end

# Función para calcular distancias
function calcular_matriz_distancias(coordenadas)
    n = length(coordenadas)
    distancias = zeros(n, n)
    
    for i in 1:n
        for j in 1:n
            if i != j
                # Usamos una aproximación Euclidiana simple
                # Para distancias geográficas más precisas necesitarías la fórmula de Haversine
                lat1, lon1 = coordenadas[i][2], coordenadas[i][3]
                lat2, lon2 = coordenadas[j][2], coordenadas[j][3]
                distancias[i,j] = sqrt((lat2 - lat1)^2 + (lon2 - lon1)^2)
            end
        end
    end
    return distancias
end

calcular_matriz_distancias (generic function with 1 method)

In [ ]:
function resolver_tsp_MTZ(distancias)
    n = size(distancias, 1)
    model = Model(HiGHS.Optimizer)
    
    # Variables binarias: x[i,j] = 1 si vamos de ciudad i a j
    @variable(model, x[1:n, 1:n], Bin)
    
    # Variables auxiliares para eliminar sub-tours (MTZ)
    @variable(model, u[1:n] >= 0)
    
    # Función objetivo: minimizar distancia total
    @objective(model, Min, sum(distancias[i,j] * x[i,j] for i in 1:n, j in 1:n if i != j))
    
    # Restricciones: cada ciudad tiene exactamente una entrada
    for j in 1:n
        @constraint(model, sum(x[i,j] for i in 1:n if i != j) == 1)
    end
    
    # Restricciones: cada ciudad tiene exactamente una salida
    for i in 1:n
        @constraint(model, sum(x[i,j] for j in 1:n if i != j) == 1)
    end
    
    # Restricciones MTZ para eliminar sub-tours
    for i in 2:n, j in 2:n
        if i != j
            @constraint(model, u[i] - u[j] + n * x[i,j] <= n - 1)
        end
    end
    
    # Límites para u 
    for i in 2:n
        @constraint(model, u[i] >= 1)
        @constraint(model, u[i] <= n)
    end
    
    # Resolver
    optimize!(model)
    
    return model, x, u
end

resolver_tsp_MTZ (generic function with 1 method)

In [ ]:
# Cargar datos
coordenadas = leer_archivo_tsp("../data/input/gr202.tsp")
println("Número de ciudades: ", length(coordenadas))

# Para usar un subconjunto, con 50 funciona bien
coordenadas = coordenadas[1:50]

# Calcular matriz de distancias
distancias = calcular_matriz_distancias(coordenadas)
println("Matriz de distancias calculada")

# Resolver TSP
model, x, u = resolver_tsp_MTZ(distancias)

# Mostrar resultados
if termination_status(model) == MOI.OPTIMAL
    println("\n¡Solución óptima encontrada!")
    println("Distancia total: ", objective_value(model))
    
    # Reconstruir el tour
    n = length(coordenadas)
    tour = Int[]
    ciudad_actual = 1
    push!(tour, ciudad_actual)
    
    for _ in 1:n-1
        for j in 1:n
            if ciudad_actual != j && value(x[ciudad_actual, j]) ≈ 1.0
                ciudad_actual = j
                push!(tour, ciudad_actual)
                break
            end
        end
    end
    
    println("\nTour óptimo:")
    for i in 1:length(tour)
        ciudad_id = coordenadas[tour[i]][1]
        lat = coordenadas[tour[i]][2]
        lon = coordenadas[tour[i]][3]
        println("$i. Ciudad $ciudad_id: ($lat, $lon)")
    end
    
else
    println("No se encontró solución óptima")
    println("Estado: ", termination_status(model))
end

Número de ciudades: 202
Matriz de distancias calculada
Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 2550 rows; 2550 cols; 12054 nonzeros; 2500 integer variables (2500 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+01]
  Cost   [8e-02, 4e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 5e+01]
Presolving model
2452 rows, 2499 cols, 11956 nonzeros  0s
2452 rows, 2499 cols, 11956 nonzeros  0s

Solving MIP model with:
   2452 rows
   2499 cols (2450 binary, 0 integer, 0 implied int., 49 continuous, 0 domain fixed)
   11956 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Obj